In [63]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:12 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:14 http://security.ubuntu.com/ubuntu focal-security/univers

In [64]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-01-19 12:22:33--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.08s   

2023-01-19 12:22:34 (10.5 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [65]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [70]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Wireless_v1_00.tsv.gz"

spark.sparkContext.addFile(url)


In [71]:
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"


spark.sparkContext.addFile(url)

In [72]:
SparkFiles.get("amazon_reviews_us_Wireless_v1_00.tsv.gz")


'/tmp/spark-a700371a-f0eb-4d0c-9cc0-f3e2e65db7e3/userFiles-5b1d1ecb-7e46-4b82-a09f-42dcfb5193d5/amazon_reviews_us_Wireless_v1_00.tsv.gz'

In [73]:
SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz")

'/tmp/spark-a700371a-f0eb-4d0c-9cc0-f3e2e65db7e3/userFiles-5b1d1ecb-7e46-4b82-a09f-42dcfb5193d5/amazon_reviews_us_Video_Games_v1_00.tsv.gz'

In [74]:
cd /tmp/spark-a700371a-f0eb-4d0c-9cc0-f3e2e65db7e3/userFiles-5b1d1ecb-7e46-4b82-a09f-42dcfb5193d5

/tmp/spark-a700371a-f0eb-4d0c-9cc0-f3e2e65db7e3/userFiles-5b1d1ecb-7e46-4b82-a09f-42dcfb5193d5


In [75]:
cd /tmp/spark-a700371a-f0eb-4d0c-9cc0-f3e2e65db7e3/userFiles-5b1d1ecb-7e46-4b82-a09f-42dcfb5193d5

/tmp/spark-a700371a-f0eb-4d0c-9cc0-f3e2e65db7e3/userFiles-5b1d1ecb-7e46-4b82-a09f-42dcfb5193d5


In [76]:
ls


amazon_reviews_us_Video_Games_v1_00.tsv.gz*  postgresql-42.2.9.jar
amazon_reviews_us_Wireless_v1_00.tsv*        spark-3.2.3-bin-hadoop2.7/
amazon_reviews_us_Wireless_v1_00.tsv.gz*     spark-3.2.3-bin-hadoop2.7.tgz


In [77]:
!gunzip amazon_reviews_us_Wireless_v1_00.tsv.gz

gzip: amazon_reviews_us_Wireless_v1_00.tsv already exists; do you wish to overwrite (y or n)? n
	not overwritten


In [78]:
ls


amazon_reviews_us_Video_Games_v1_00.tsv.gz*  postgresql-42.2.9.jar
amazon_reviews_us_Wireless_v1_00.tsv*        spark-3.2.3-bin-hadoop2.7/
amazon_reviews_us_Wireless_v1_00.tsv.gz*     spark-3.2.3-bin-hadoop2.7.tgz


In [79]:
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Wireless_v1_00.tsv"), sep="\t", header=True)

df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16414143|R3W4P9UBGNGH1U|B00YL0EKWE|     852431543|LG G4 Case Hard T...|        Wireless|          2|            1|          3|   N|                Y|Looks good, funct...|2 issues  -  Once...| 2015-08-31|
|         US|   50800750|R15V54KBMTQWAY|B00XK95RPQ|     516894650|Selfie Stick Fibl...|        Wireless|          4|    

In [80]:
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), sep="\t", header=True)

df.show()


+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [81]:
# Get the number of rows in the DataFrame.

df.count()


1785997

In [82]:
df = df.dropna()
print(df.count())

1785886


# Transform the Data

In [83]:
# Print schema
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: string (nullable = true)
 |-- total_votes: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [84]:

from pyspark.sql.types import TimestampType, IntegerType

df = df.withColumn("customer_id", df["customer_id"].cast(IntegerType()))\
       .withColumn("product_parent", df["product_parent"].cast(IntegerType()))\
       .withColumn("review_date", df["review_date"].cast(TimestampType()))\
       .withColumn("star_rating", df["star_rating"].cast(IntegerType()))\
       .withColumn("helpful_votes", df["helpful_votes"].cast(IntegerType()))\
      .withColumn("total_votes", df["total_votes"].cast(IntegerType()))\

# Print schema
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: timestamp (nullable = true)



## Create the "review_id_table".

In [85]:
from pyspark.sql.functions import to_date
# Create the "review_id_df" DataFrame with the appropriate columns and data types.'
review_id_df = df.select(["review_id","customer_id","product_id", "product_parent","review_date"])
review_id_df.show()


+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
| RTIS3L2M1F5SM|   12039526|B001CXYMFS|     737716809|2015-08-31 00:00:00|
| R1ZV7R40OLHKD|    9636577|B00M920ND6|     569686175|2015-08-31 00:00:00|
|R3BH071QLH8QMC|    2331478|B0029CSOD2|      98937668|2015-08-31 00:00:00|
|R127K9NTSXA2YH|   52495923|B00GOOSV98|      23143350|2015-08-31 00:00:00|
|R32ZWUXDJPW27Q|   14533949|B00Y074JOM|     821342511|2015-08-31 00:00:00|
|R3AQQ4YUKJWBA6|    2377552|B002UBI6W6|     328764615|2015-08-31 00:00:00|
|R2F0POU5K6F73F|   17521011|B008XHCLFO|      24234603|2015-08-31 00:00:00|
|R3VNR804HYSMR6|   19676307|B00BRA9R6A|     682267517|2015-08-31 00:00:00|
| R3GZTM72WA2QH|     224068|B009EPWJLA|     435241890|2015-08-31 00:00:00|
| RNQOY62705W1K|   48467989|B0000AV7GB|     256572651|2015-08-31 00:00:00|
|R1VTIA3JTYBY02|     1065

## Create the "products" Table

In [86]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 

products_df = df.select(["product_id","product_title"])
products_df = products_df.dropDuplicates()
products_df.show()


+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00CJ7IUI6|The Elder Scrolls...|
|B00DHF39KS|Wolfenstein: The ...|
|B00MUTAVH6|Under Night In-Bi...|
|B001AZSEUW|              Peggle|
|B00KVOVBGM|PlayStation 4 Con...|
|B00O9VGH4Y|USPRO&reg; Headph...|
|B004OQNZY4|Phineas and Ferb:...|
|B00ZLN980O|Donop seablue 2.4...|
|B002L8W5V6|Dotop Nintendo Ga...|
|B007AJZ5PY|Nyko Game Case fo...|
|B000AOEU2K|Fire Emblem: Path...|
|B000H8BW7U|Tanarus (PC) (Com...|
|B013RADQOQ|Susenstone® 2400D...|
|B00KQXKUJ2|FIFA 15 (Ultimate...|
|B006W41X2C|Turtle Beach - Ea...|
|B000KCX9M4|Grand Theft Auto:...|
|B00YT90JWC|Red Wii Mini Cons...|
|B0096KG6A8|Wii U Super Mario...|
|B00L6AVLB0|World of Tanks-X3...|
|B000IMYKQ0|Wii Nunchuk Contr...|
+----------+--------------------+
only showing top 20 rows



## Create the "customers" Table

In [87]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 

from pyspark.sql.functions import desc

customers_df = df.groupby("customer_id").agg({"customer_id":"count"})
customers_df = customers_df.orderBy(desc("count(customer_id)"))
customers_df = customers_df.withColumnRenamed("count(customer_id)", "customer_count") 
customers_df.show()


+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   52759271|           909|
|   42418272|           856|
|   45838740|           839|
|   24846840|           614|
|   24594001|           460|
|   40955995|           459|
|   48278407|           439|
|   27364030|           361|
|   30678701|           326|
|   50648042|           301|
|   14539589|           288|
|   10541465|           271|
|   38752775|           263|
|   43092063|           257|
|   45261621|           235|
|   44603910|           233|
|   14886512|           233|
|   37687227|           226|
|   10075230|           218|
|   24476167|           214|
+-----------+--------------+
only showing top 20 rows



## Create the "vine_table".

In [88]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 


vine_table_df = df.select(["review_id","star_rating","helpful_votes", "total_votes","vine"])

# # Filter for only columns from Amazon's Vine program
# vine_table_df = vine_table_df.filter(col("vine")  == "Y")
vine_table_df.show() 


+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RTIS3L2M1F5SM|          5|            0|          0|   N|
| R1ZV7R40OLHKD|          5|            0|          0|   N|
|R3BH071QLH8QMC|          1|            0|          1|   N|
|R127K9NTSXA2YH|          3|            0|          0|   N|
|R32ZWUXDJPW27Q|          4|            0|          0|   N|
|R3AQQ4YUKJWBA6|          1|            0|          0|   N|
|R2F0POU5K6F73F|          5|            0|          0|   N|
|R3VNR804HYSMR6|          5|            0|          0|   N|
| R3GZTM72WA2QH|          5|            0|          0|   N|
| RNQOY62705W1K|          4|            0|          0|   N|
|R1VTIA3JTYBY02|          5|            0|          0|   N|
|R29DOU8791QZL8|          1|            0|          0|   N|
|R15DUT1VIJ9RJZ|          2|            0|          0|   N|
|R3IMF2MQ3OU9ZM|          4|            

# Load

In [ ]:
mode = "append"
jdbc_url="jdbc:postgresql://<endpoint>:5432/my_data_class_db"
config = {"user":"postgres", "password": "<password>", "driver":"org.postgresql.Driver"}

In [89]:
mode = "append"
jdbc_url="jdbc:postgresql://mypostgresdb.cycvkytkmcts.ap-northeast-1.rds.amazonaws.com:5432/my_data_class_db"
config = {"user":"root", "password": "prisha2882", "driver":"org.postgresql.Driver"}

In [90]:
# Write review_id_df to table in RDS

review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)


ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/content/spark-3.2.3-bin-hadoop2.7/python/lib/py4j-0.10.9.5-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/content/spark-3.2.3-bin-hadoop2.7/python/lib/py4j-0.10.9.5-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored

In [ ]:
# Write products_df to table in RDS


In [ ]:
# Write customers_df to table in RDS


In [ ]:
# Write vine_df to table in RDS
